In [14]:
import pandas as pd
import sklearn as skl
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from imageio import mimwrite

from sklearn.preprocessing import StandardScaler

import cartopy
import cartopy.crs as ccrs

from shapely.geometry import Polygon
from shapely.ops import cascaded_union

from datetime import date

import tensorflow as tf

In [15]:
nytimes_data = pd.read_csv('~/nytimes/covid-19-data/us-counties.csv')

nytimes_data = nytimes_data.iloc[np.where(np.isfinite(nytimes_data[['fips']].values))[0]]

nytimes_data['predecessor_date'] = (
    [date.fromisoformat(x).toordinal() - 1 for x in nytimes_data['date']])

In [17]:
nytimes_data.shape

(210334, 7)

In [18]:
## From US Census Bureau records
county_reader = cartopy.io.shapereader.Reader('./cb_2018_us_county_500k.shp')
counties = list(county_reader.records())
county_indices = pd.DataFrame({
    'GEOID' : [float(counties[i].attributes['GEOID']) for i in range(len(counties))],
    'index' : [i for i in range(len(counties))]})

road_reader = cartopy.io.shapereader.Reader('./tl_2019_us_primaryroads.shp')
roads = list(road_reader.records())

interstates = [
    road for road in roads if (road.attributes['RTTYP'] == 'I')]
interstate_names = np.unique([(road.attributes['FULLNAME']) 
                              for road in interstates])

interstates_by_name = (
    {name : 
     [road.geometry 
      for road in roads
      if road.attributes['FULLNAME'] == name]
     for name in interstate_names})

county_contents = {}

for county in counties:
    county_GEOID = int(county.attributes['GEOID'])
    
    intersecting_interstates = [
        interstate_name
        for interstate_name in interstate_names
        if any(
            segment_geometry
            for segment_geometry in interstates_by_name[interstate_name]
            if segment_geometry.intersects(county.geometry)

        )]
    
    if len(intersecting_interstates) > 0:
        county_contents[county_GEOID] = intersecting_interstates

interstate_contents = {}

for interstate_name in interstates_by_name:
    interstate_segments = interstates_by_name[interstate_name]

    intersecting_counties = [
        int(county.attributes['GEOID'])
        for county in counties
        if any(
            segment_geometry
            for segment_geometry in interstate_segments
            if segment_geometry.intersects(county.geometry))]
    
    interstate_contents[interstate_name] = intersecting_counties

In [19]:
def get_county_interstate_counts(date):
    restricted_data = nytimes_data[nytimes_data.date == date]

    restricted_data.columns = ['GEOID' if x == 'fips'
                               else x
                               for x in restricted_data.columns]

    reader = cartopy.io.shapereader.Reader('./tl_2019_us_primaryroads.shp')
    roads = list(reader.records())

    interstates = [road for road in roads if (road.attributes['RTTYP'] == 'I')]
    interstate_names = np.unique([(road.attributes['FULLNAME']) 
                                  for road in roads 
                                  if (road.attributes['RTTYP'] == 'I')])

    interstates_by_name = (
        {name : 
         [road.geometry 
          for road in roads
          if road.attributes['FULLNAME'] == name]
         for name in interstate_names})

    extended_interstate_keys = pd.DataFrame(
        data={'interstate_name' : 
              np.concatenate([np.repeat(x, len(interstate_contents[x]))
                              for x in interstate_contents]),
              'GEOID' :
              np.concatenate([np.array(interstate_contents[x]).astype(np.intc)
                              for x in interstate_contents])})

    interstate_county_counts = (
        restricted_data.merge(extended_interstate_keys, on='GEOID').
        groupby(['interstate_name']).
        sum())

    extended_county_keys = pd.DataFrame(
        data={
            'GEOID' :
            np.concatenate([np.repeat(x, len(county_contents[x])).astype(np.intc)
                            for x in county_contents]),
            'interstate_name' :
            np.concatenate([np.array(county_contents[x])
                            for x in county_contents])})

    county_traffic_counts = (
        extended_county_keys.merge(interstate_county_counts, on='interstate_name').
        groupby(['GEOID_x']).
        sum())
    
    return county_traffic_counts

county_traffic_counts_dict_base = {
    date :
    get_county_interstate_counts(date)
    for date in np.unique(nytimes_data.date)}

In [20]:
## Read the inbound and outbound work travel CSV

inbound_travel = pd.read_csv('./inbound.csv')

reduced_inbound_travel_base = pd.DataFrame(
    {
        'fips' : (
            1000 * inbound_travel['State FIPS Code'] +
            inbound_travel['County FIPS Code']),
        'dest_GEOID' : (
            1000 * inbound_travel['State FIPS Code.1'] + 
            inbound_travel['County FIPS Code.1']),
        'flow' : (
            [int(x.replace(',', '')) 
             for x 
             in inbound_travel['Workers in Commuting Flow']])
    }
).groupby('dest_GEOID').sum()

reduced_inbound_travel = pd.DataFrame({
    'fips' : [int(GEOID)
         for GEOID, row 
         in reduced_inbound_travel_base.iterrows()],
    'flow' : [row.flow
         for GEOID, row 
         in reduced_inbound_travel_base.iterrows()]})

In [21]:
state_shelter_in_place_dates_base = pd.read_csv('./state_shelter_in_place_dates.csv')

state_shelter_in_place = pd.DataFrame({
    'state' : state_shelter_in_place_dates_base['State'],
    'shelter_date_present' : [
        int(type(x) == str)
        for x 
        in state_shelter_in_place_dates_base['Shelter in place']],
    'shelter_date' : [
        float(date.fromisoformat(x).toordinal()) if type(x) == str 
        else np.NaN 
        for x 
        in state_shelter_in_place_dates_base['Shelter in place']]})

In [22]:
def merge_inbound_travel_counts(record_date):
    merged_inbound_travel = (
        reduced_inbound_travel.merge(
            nytimes_data[nytimes_data.date == record_date], 
            on='fips'))
    
    county_inbound_travel_counts = (
        merged_inbound_travel.merge(
            state_shelter_in_place,
            on='state',
            how='left')
    )
    
    county_inbound_travel_counts['latency_in_place'] = [
        (date.fromisoformat(record_date).toordinal() - x)
        if (np.isfinite(x) and
            x <= date.fromisoformat(record_date).toordinal())
        else -1
        for x
        in county_inbound_travel_counts[ 'shelter_date']]
    
    final_inbound_travel_counts = (
        county_inbound_travel_counts.merge(
            county_traffic_counts_dict_base[record_date],
            left_on='fips',
            right_on='GEOID_x',
            how='left'))
    
    final_inbound_travel_counts['ordinal_date'] = (
        [date.fromisoformat(record_date).toordinal()] * 
        final_inbound_travel_counts.shape[0])
    
    output_inbound_travel_counts = (
        final_inbound_travel_counts.merge(
            nytimes_data,
            left_on=['ordinal_date', 'fips'],
            right_on=['predecessor_date', 'fips']))
    
    return output_inbound_travel_counts

merge_inbound_travel_counts_dict_base = {
    date :
    merge_inbound_travel_counts(date)
    for date in np.unique(nytimes_data.date)}

In [23]:
all_train_data_base = pd.concat([
    merge_inbound_travel_counts_dict_base[x] 
    for x in merge_inbound_travel_counts_dict_base])

train_data_base_columns = [
    'flow',
    'cases_x',
    'deaths_x',
    'shelter_date_present',
    'latency_in_place',
    'cases_y',
    'deaths_y',
    'cases',
    'deaths',
    'fips',
]

all_train_data_columns = [
    'intercounty_flow',
    'local_cases',
    'local_deaths',
    'sheltering_in_place',
    'ndays_sheltering_in_place',
    'interstate_borne_cases',
    'interstate_borne_deaths',
    'target_cases',
    'target_deaths',
    'county_GEOID'
]

all_train_data_frame = all_train_data_base[train_data_base_columns]
all_train_data_frame.columns = all_train_data_columns

all_train_data = all_train_data_frame.values
all_train_data[~np.isfinite(all_train_data[:, 3]), 3] = 0
all_train_data[~np.isfinite(all_train_data[:, 5]), 5] = 0
all_train_data[~np.isfinite(all_train_data[:, 6]), 6] = 0

train_data_scaler = StandardScaler()
all_train_data[:, :-3] = train_data_scaler.fit_transform(all_train_data[:, :-3])

In [24]:
predictor_input = tf.keras.Input(shape=(all_train_data.shape[1] - 3,), )
predictor_inner = tf.keras.layers.Dense(10, activation='relu')(predictor_input)
predictor_inner = tf.keras.layers.Dense(10, activation='relu')(predictor_inner)
predictor_output = tf.keras.layers.Dense(1, activation='linear')(predictor_inner)

predictor_model = tf.keras.Model(inputs=predictor_input, outputs=predictor_output)
predictor_model.summary()

predictor_model.compile(tf.keras.optimizers.Nadam(lr=1e-3), 
                        loss=tf.keras.losses.MeanAbsoluteError(),
                        metrics=['mse', 'mae'],
                       )

predictor_history = predictor_model.fit(
    all_train_data[:, :-3], 
    all_train_data[:, -3],
    epochs=50,
    verbose=2,
    batch_size=100,)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                80        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Train on 207348 samples
Epoch 1/50
207348/207348 - 11s - loss: 206.9688 - mse: 1895686.0000 - mae: 206.9687
Epoch 2/50
207348/207348 - 8s - loss: 55.3072 - mse: 76867.0938 - mae: 55.3072
Epoch 3/50
207348/207348 - 8s - loss: 33.5282 - mse: 31711.8008 - mae: 3

In [10]:
results = predictor_model.predict(all_train_data[:, :-3])

In [11]:
y = pd.DataFrame(
    np.hstack([results, 
               np.array(all_train_data[:, -3]).reshape(-1, 1),
               np.array(all_train_data[:, -1]).reshape(-1, 1), 
               all_train_data_base['ordinal_date'].values.reshape(-1, 1)
              ]))
y.columns = ['predicted_count', 'actual_count', 'GEOID', 'ordinal_date']

In [12]:
def plot_county_cases(result_array, date_string):
    restricted_result_array = pd.DataFrame(
        result_array.loc()[
            date.fromisoformat(date_string).toordinal() == result_array.iloc()[:, 3],
            :]).merge(county_indices).values
    
    central_lat = 37.5
    central_lon = -96
    extent = [-120, -70, 23, 50.5]
    central_lon = np.mean(extent[:2])
    central_lat = np.mean(extent[2:])

    fig, ax = plt.subplots(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
    ax.set_extent(extent)

    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAND, edgecolor='black')
    ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
    ax.add_feature(cartopy.feature.BORDERS)
    ax.add_feature(
        cartopy.feature.ShapelyFeature(
            [counties[int(restricted_result_array[int(i), -1])].geometry 
             for i in range(restricted_result_array.shape[0])], 
            cartopy.crs.PlateCarree(),
            color='pink'))

    ax.add_feature(cartopy.feature.STATES, edgecolor='lightgrey')
    ax.add_feature(
        cartopy.feature.ShapelyFeature(
            [interstate.geometry for interstate in interstates],
            cartopy.crs.PlateCarree(),
            color='grey'))

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.close()
    
    return image

county_cases = [
    plot_county_cases(y, image_date) 
    for image_date in np.unique(nytimes_data.date)[:-1]
]

mimwrite('./county_cases.gif', 
         county_cases,
         fps=5,
         subrectangles=True,
         loop=1)

In [13]:
def plot_county_counts(result_array, date_string):
    restricted_result_array = pd.DataFrame(
        result_array.loc()[
            date.fromisoformat(date_string).toordinal() == result_array.iloc()[:, 3],
            :]).merge(county_indices).values
        
    extent = [-125, -65, 23, 50.5]
    central_lon = np.mean(extent[:2])
    central_lat = np.mean(extent[2:])
    
    main_proj = cartopy.crs.PlateCarree()

    fig, ax = plt.subplots(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
    ax.set_extent(extent)
    ax.set_aspect('auto')

    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAND, edgecolor='black')
    ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
    ax.add_feature(cartopy.feature.BORDERS)

    centroids = [
        counties[int(restricted_result_array[int(i), -1])].
        geometry.
        centroid.
        xy
        for i in range(restricted_result_array.shape[0])]
    
    diffs = [np.abs(restricted_result_array[int(i), 0] - restricted_result_array[int(i), 1] )
        for i in range(restricted_result_array.shape[0])]

    for (centroid, diff) in zip(centroids, diffs):
        x, y = centroid
        radius = diff * 0.05
        ax.add_patch(patches.Circle(xy=[x[0], y[0]], 
                                    radius=radius,                           
                                    color='b', 
                                    alpha=0.3, 
                                    transform=main_proj))

    ax.add_feature(cartopy.feature.STATES, edgecolor='lightgrey')
            
    ax.add_feature(
        cartopy.feature.ShapelyFeature(
            [interstate.geometry for interstate in interstates],
            cartopy.crs.PlateCarree(),
            color='grey'))            
            
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close()
    
    return image

county_counts = [
    plot_county_counts(y, image_date) 
    for image_date in np.unique(nytimes_data.date)[:-1]
]

mimwrite('./county_counts.gif',
         county_counts, 
         fps=5,
         subrectangles=True,
         loop=1)

In [25]:
def plot_county_counts(result_array, date_string):
    restricted_result_array = pd.DataFrame(
        result_array.loc()[
            date.fromisoformat(date_string).toordinal() == result_array.iloc()[:, 3],
            :]).merge(county_indices).values
        
    extent = [-125, -65, 23, 50.5]
    central_lon = np.mean(extent[:2])
    central_lat = np.mean(extent[2:])
    
    main_proj = cartopy.crs.PlateCarree()

    fig, ax = plt.subplots(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
    ax.set_extent(extent)
    ax.set_aspect('auto')

    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAND, edgecolor='black')
    ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
    ax.add_feature(cartopy.feature.BORDERS)

    centroids = [
        counties[int(restricted_result_array[int(i), -1])].
        geometry.
        centroid.
        xy
        for i in range(restricted_result_array.shape[0])]
    
    counts = [restricted_result_array[int(i), 1]
        for i in range(restricted_result_array.shape[0])]

    for (centroid, count) in zip(centroids, counts):
        x, y = centroid
        radius = count * 0.05
        ax.add_patch(patches.Circle(xy=[x[0], y[0]], 
                                    radius=radius,                           
                                    color='b', 
                                    alpha=0.3, 
                                    transform=main_proj))

    ax.add_feature(cartopy.feature.STATES, edgecolor='lightgrey')
            
    ax.add_feature(
        cartopy.feature.ShapelyFeature(
            [interstate.geometry for interstate in interstates],
            cartopy.crs.PlateCarree(),
            color='grey'))            
            
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close()
    
    return image

county_counts = [
    plot_county_counts(y, image_date) 
    for image_date in np.unique(nytimes_data.date)[:-1]
]

mimwrite('./county_counts.gif',
         county_counts, 
         fps=5,
         subrectangles=True,
         loop=1)